In [1]:
# Алексеев Д.П. (DSU-4,FEML-8)
# Домашнее задание к лекции «Функции потерь и оптимизация» (#3).

# Задание:
# Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
# Реализовать самостоятельно логистическую регрессию
# Обучить ее методом градиентного спуска:
# -Методом nesterov momentum
# -Методом rmsprop

# Дополнительное задание *
# В качестве dataset’а взять Iris, оставив 2 класса:
# Iris Versicolor
# Iris Virginica

# Примечание: если я правильно понял задание, 
# то необходимо реализовать логистическую регрессию для предсказаний классов 'Iris Versicolor' и 'Iris Virginica'?

In [73]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

In [76]:
# загружаем датасет с ирисами Фишера (сразу как датафрейм)
iris = datasets.load_iris(as_frame = True).frame
iris
# 4 входящих (описательных) признака | sepal - чашелистик, petal - лепесток
# значениe целевой переменной target (класс цветка): 'setosa' =0, 'versicolor'=1, 'virginica'=2

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [77]:
# по условию задачи, нам нужна классификация только 'versicolor' и 'virginica' ('versicolor'=1, 'virginica'=2)
# Следовательно, строки с классом=0, т.е. 'setosa', можно выкинуть.
iris = iris[iris['target'] != 0]
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [95]:
X = iris[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
50,7.0,3.2,4.7,1.4
51,6.4,3.2,4.5,1.5
52,6.9,3.1,4.9,1.5
53,5.5,2.3,4.0,1.3
54,6.5,2.8,4.6,1.5
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [96]:
# https://ru.wikipedia.org/wiki/Логистическая_регрессия
# Логистическая регрессия - это модель, используемая для прогнозирования вероятности возникновения некоторого события 
# путём его сравнения с логистической кривой. В данном случае ключевую роль играет сигмоид (логистическая кривая).
# Эта регреcсия выдаёт ответ в виде вероятности бинарного события (1 или 0).

# Т.к. логистическая регрессия работает с бинарной классификацией, то приведем целевую переменную к бинарному виду:
# 'versicolor'= 1-1 =0, 'virginica'= 2-1 = 1

Y = iris[['target']] - 1
Y

,target
50,0
51,0
52,0
53,0
54,0
...,...
145,1
146,1
147,1
148,1


In [122]:
# по теоретическим описаниям логистической регрессии возникли сложности с написанием работающего кода, 
# поэтому алгоритм и частично код пришлось позаимствовать по ссылке...
# https://pythobyte.com/logistic-regression-from-scratch-ae373d5d/

# определяем функцию расчета сигмоида
def sigmoid(x, weight):
    z = np.dot(x, weight) 
    return 1 / (1 + np.exp(-z))

# определяем функцию потерь
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

# функция расчета шага градиентного спуска
def gradient_descent(x, h, y):
    return np.dot(x.T, (h - y)) / y.shape[0]

# непосредственно функция оптимизации методом градиентного спуска
def fit(x, y, lr , iterations):
    weight = np.zeros(x.shape[1])
    for i in range(iterations):
        sigma = sigmoid(x, weight)
            
        loss = loss(sigma, y)

        dW = gradient_descent(x , sigma, y)
            
        #обновляем веса
        weight -= lr * dW

    return print('fitted successfully to data')
    
# функция предсказания вероятности принадлежности к классу 0 или 1 с пороговым значением вероятности treshold (обычно 0.5)
def predict(x_new , treshold):
    x_new = np.concatenate((intercept, x_new), axis=1)
    result = sigmoid(x_new, weight)
    result = result >= treshold
    y_pred = np.zeros(result.shape[0])
    for i in range(len(y_pred)):
        if result[i] == True: 
            y_pred[i] = 1
        else:
            continue
                
    return y_pred

In [124]:
result = fit(X, Y, 0.1, 50)
result

UnboundLocalError: local variable 'loss' referenced before assignment

In [3]:
# метод Nesterov Momentum
# def NM(X, gamma, lr=0.25):
#     Y = []
#     v = 0
#     for i in range(len(X)):
#         v = gamma*v + lr*(linear_interpolation(X, i+gamma*v) if i+gamma*v < len(X) else 0)
#         Y.append(v)
#     return np.asarray(Y)

In [4]:
# метод RMSProp
# def RMSProp(X, gamma, lr=0.25, eps=0.00001):
#     Y = []
#     EG = 0
#     for x in X:
#         EG = gamma*EG + (1-gamma)*x*x
#         v = lr/np.sqrt(EG + eps)*x
#         Y.append(v)
#     return np.asarray(Y)